# PSI and Traffic in Regional level in real time

In [68]:
import requests
import io
import pandas as pd
import zipfile

# @Author: Harpal
# Define AirQualityData class for PSI data (Regional level in real time)
class AirQualityData:
    def __init__(self):
        self.api_url = "https://api.data.gov.sg/v1/environment/psi"
        
    def get_psi_data(self):
        """Fetch real-time PSI data for Singapore from the NEA API."""
        try:
            response = requests.get(self.api_url)
            response.raise_for_status()
            data = response.json()
            psi_data = {}
            for region in data['region_metadata']:
                name = region['name']
                psi_data[name] = data['items'][0]['readings']
            return psi_data
        except requests.exceptions.RequestException as e:
            print(f"Error fetching PSI data: {e}")
            return None

# @Author: Lucy
# Define VehicleTypeData class
# TODO Vehicle population was moved to static datasets as zipped files. There are more vehicle populations by different categorical ways
# TODO Vehicle Type is static to monthly level withour regional information
class VehicleTypeData:
    def __init__(self, zip_url):
        self.zip_url = zip_url
        
    def get_vehicle_distribution(self):
        """Fetch vehicle population by fuel type from a ZIP containing CSV file."""
        try:
            # Download the ZIP file
            response = requests.get(self.zip_url)
            response.raise_for_status()
            
            # Convert the downloaded content into a byte stream
            zip_file_bytes = io.BytesIO(response.content)
            
            # Open the ZIP file
            with zipfile.ZipFile(zip_file_bytes, 'r') as zip_ref:
                # List all files in the ZIP archive
                file_names = zip_ref.namelist()
                # Only one CSV file within the ZIP archive
                csv_file_name = [name for name in file_names if name.endswith('.csv')][0]
                
                # Read the CSV file from the ZIP archive
                with zip_ref.open(csv_file_name) as csv_file:
                    df = pd.read_csv(csv_file)

            # Process CSV data and calculate vehicle distribution by fuel type
            vehicle_distribution = {}
            for index, row in df.iterrows():
                try:
                    fuel_type = row['type']
                    # Remove commas from the number string and convert to int
                    number = int(row['number'].replace(',', ''))
                    if fuel_type not in vehicle_distribution:
                        vehicle_distribution[fuel_type] = 0
                    vehicle_distribution[fuel_type] += number
                except ValueError:
                    print(f"Skipping row due to invalid number format: {row}")
                    # Skip the current row if conversion fails
                    continue
            return vehicle_distribution
        except requests.exceptions.RequestException as e:
            print(f"Error fetching vehicle data: {e}")
            return None
        except zipfile.BadZipFile as e:
            print(f"Error with zip file: {e}")
            return None

# @Author: Harpal
# Define TrafficCongestionData class for TomTom API (more granular, neighborhood level)
class TrafficCongestionData:
    def __init__(self, api_key):
        self.api_url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
        self.api_key = api_key
    
    def get_traffic_data(self, coordinates):
        """Fetch traffic congestion data for given coordinates."""
        try:
            response = requests.get(
                f"{self.api_url}?point={coordinates}&unit=KMPH&key={self.api_key}"
            )
            response.raise_for_status()
            data = response.json()
            return data['flowSegmentData']
        except requests.exceptions.RequestException as e:
            print(f"Error fetching traffic data: {e}")
            return None

# Neighborhood mapping by region
neighborhoods_by_region = {
    "north": ["Woodlands", "Sembawang", "Yishun"],
    "south": ["Sentosa", "Marina Bay", "Bukit Merah"],
    "east": ["Tampines", "Bedok", "Pasir Ris"],
    "west": ["Jurong East", "Jurong West", "Bukit Batok"],
    "central": ["Orchard", "Toa Payoh", "Novena"]
}

# Function to map coordinates for specific neighborhoods
neighborhood_coordinates = {
    "Woodlands": "1.436,103.786",
    "Sembawang": "1.449,103.818",
    "Yishun": "1.429,103.836",
    "Sentosa": "1.249,103.830",
    "Marina Bay": "1.290,103.859",
    "Bukit Merah": "1.286,103.820",
    "Tampines": "1.345,103.944",
    "Bedok": "1.321,103.927",
    "Pasir Ris": "1.373,103.949",
    "Jurong East": "1.332,103.743",
    "Jurong West": "1.352,103.717",
    "Bukit Batok": "1.356,103.762",
    "Orchard": "1.304,103.831",
    "Toa Payoh": "1.332,103.847",
    "Novena": "1.320,103.844"
}

# Main function to integrate neighborhood-level data
def analyze_vehicle_traffic_air_quality_neighborhood(vehicle_api_key, traffic_api_key):
    # 1. Fetch PSI data (still region-level)
    air_quality_data = AirQualityData()
    psi_info = air_quality_data.get_psi_data()
    print("Pollutant Standards Index (PSI) Data (Region-Level):")
    print(psi_info)
    
    # 2. Fetch vehicle type data (region-level)
    zip_url = 'https://datamall.lta.gov.sg/content/dam/datamall/datasets/Facts_Figures/Vehicle%20Population/Monthly%20Motor%20Vehicle%20Population%20Statistics%20by%20Type%20of%20Fuel%20Used.zip'
    vehicle_data = VehicleTypeData(zip_url)
    vehicle_info = vehicle_data.get_vehicle_distribution()
    print("\nVehicle Distribution by Fuel Type (Region-Level):")
    print(vehicle_info)
    
    # 3. Fetch traffic congestion data for each neighborhood
    traffic_data = TrafficCongestionData(traffic_api_key)
    print("\nNeighborhood-Level Traffic Data:")
    
    results = []
    for region, neighborhoods in neighborhoods_by_region.items():
        for neighborhood in neighborhoods:
            coordinates = neighborhood_coordinates[neighborhood]
            traffic_info = traffic_data.get_traffic_data(coordinates)
            psi_values = psi_info.get(region, {})
            
            # Collect and organize the results
            result = {
                "region": region,
                "neighborhood": neighborhood,
                "traffic_data": traffic_info,
                "psi_data": psi_values,
                "vehicle_distribution": vehicle_info
            }
            results.append(result)
            
            # print(f"\nRegion: {region}, Neighborhood: {neighborhood}")
            # print(f"Traffic Data: {traffic_info}")
            # print(f"PSI Data: {psi_values}")
    return results

In [69]:
vehicle_api_key = "heUwIZj4amMaPA3YVxH5D5o8HPU5oMm1"
traffic_api_key = "heUwIZj4amMaPA3YVxH5D5o8HPU5oMm1"

# Run the neighborhood-level analysis
data_df = analyze_vehicle_traffic_air_quality_neighborhood(vehicle_api_key, traffic_api_key)

Pollutant Standards Index (PSI) Data (Region-Level):
{'west': {'o3_sub_index': {'west': 4, 'east': 6, 'central': 6, 'south': 3, 'north': 4}, 'pm10_twenty_four_hourly': {'west': 21, 'east': 20, 'central': 25, 'south': 18, 'north': 28}, 'pm10_sub_index': {'west': 21, 'east': 20, 'central': 25, 'south': 18, 'north': 28}, 'co_sub_index': {'west': 6, 'east': 3, 'central': 5, 'south': 3, 'north': 6}, 'pm25_twenty_four_hourly': {'west': 13, 'east': 10, 'central': 12, 'south': 10, 'north': 12}, 'so2_sub_index': {'west': 3, 'east': 2, 'central': 2, 'south': 2, 'north': 1}, 'co_eight_hour_max': {'west': 1, 'east': 0, 'central': 0, 'south': 0, 'north': 1}, 'no2_one_hour_max': {'west': 27, 'east': 28, 'central': 36, 'south': 17, 'north': 25}, 'so2_twenty_four_hourly': {'west': 5, 'east': 4, 'central': 3, 'south': 3, 'north': 2}, 'pm25_sub_index': {'west': 52, 'east': 41, 'central': 48, 'south': 40, 'north': 51}, 'psi_twenty_four_hourly': {'west': 52, 'east': 41, 'central': 48, 'south': 40, 'north'

In [65]:
data_df[0]['traffic_data']

{'frc': 'FRC2',
 'currentSpeed': 33,
 'freeFlowSpeed': 40,
 'currentTravelTime': 1012,
 'freeFlowTravelTime': 835,
 'confidence': 0.996708,
 'roadClosure': False,
 'coordinates': {'coordinate': [{'latitude': 1.411767381776689,
    'longitude': 103.75568611634799},
   {'latitude': 1.411830394554422, 'longitude': 103.75569013966151},
   {'latitude': 1.411889385238378, 'longitude': 103.75569952739306},
   {'latitude': 1.411989937537196, 'longitude': 103.75570757402011},
   {'latitude': 1.412091914322451, 'longitude': 103.75572098506518},
   {'latitude': 1.412259501470095, 'longitude': 103.75573707831927},
   {'latitude': 1.412584117740664, 'longitude': 103.7557598770959},
   {'latitude': 1.412679307221847, 'longitude': 103.75576658261843},
   {'latitude': 1.412850916417506, 'longitude': 103.75576658261843},
   {'latitude': 1.41287245135961, 'longitude': 103.75576658261843},
   {'latitude': 1.412968981525695, 'longitude': 103.75576255930491},
   {'latitude': 1.413068193081101, 'longitude':

# Vehicle in National level in monthly time
## TODO Find relation with PSI in time aspect

In [70]:
data_df[0]['psi_data']

{'o3_sub_index': {'west': 4, 'east': 6, 'central': 6, 'south': 3, 'north': 4},
 'pm10_twenty_four_hourly': {'west': 21,
  'east': 20,
  'central': 25,
  'south': 18,
  'north': 28},
 'pm10_sub_index': {'west': 21,
  'east': 20,
  'central': 25,
  'south': 18,
  'north': 28},
 'co_sub_index': {'west': 6, 'east': 3, 'central': 5, 'south': 3, 'north': 6},
 'pm25_twenty_four_hourly': {'west': 13,
  'east': 10,
  'central': 12,
  'south': 10,
  'north': 12},
 'so2_sub_index': {'west': 3, 'east': 2, 'central': 2, 'south': 2, 'north': 1},
 'co_eight_hour_max': {'west': 1,
  'east': 0,
  'central': 0,
  'south': 0,
  'north': 1},
 'no2_one_hour_max': {'west': 27,
  'east': 28,
  'central': 36,
  'south': 17,
  'north': 25},
 'so2_twenty_four_hourly': {'west': 5,
  'east': 4,
  'central': 3,
  'south': 3,
  'north': 2},
 'pm25_sub_index': {'west': 52,
  'east': 41,
  'central': 48,
  'south': 40,
  'north': 51},
 'psi_twenty_four_hourly': {'west': 52,
  'east': 41,
  'central': 48,
  'south': 4

In [77]:
import folium
import pandas as pd

# Create a Folium map object centered at Singapore's coordinates
singapore_map = folium.Map(location=[1.3521, 103.8198], zoom_start=11)

# Iterate through data_df to add a marker for each neighborhood
for data in data_df:
    neighborhood = data['neighborhood']  # Get the neighborhood name
    region = data['region']  # Get the region name
    traffic_speed = data['traffic_data']['currentSpeed']  # Get current traffic speed
    free_flow_speed = data['traffic_data']['freeFlowSpeed']  # Get free flow speed
    pm25 = data['psi_data']['pm25_twenty_four_hourly'][region]  # Get PM2.5 data
    
    # Get the first coordinate point (latitude and longitude)
    # TODO not match somehow here, using the predefined coodinates
    lat = data['traffic_data']['coordinates']['coordinate'][0]['latitude']
    lon = data['traffic_data']['coordinates']['coordinate'][0]['longitude']
    
    # Create popup content to display traffic and PSI information
    popup_content = f"""
    <strong>Region:</strong> {region}<br>
    <strong>Neighborhood:</strong> {neighborhood}<br>
    <strong>Current Speed:</strong> {traffic_speed} KMPH<br>
    <strong>Free Flow Speed:</strong> {free_flow_speed} KMPH<br>
    <strong>PM2.5:</strong> {pm25} µg/m³
    """
    
    # Add a marker to the map with a popup showing the traffic and PSI data
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_content, max_width=300),
        tooltip=f"{neighborhood} ({region})"
    ).add_to(singapore_map)

# Display the map in Jupyter Notebook
singapore_map
